In [29]:
import pandas as pd
from kipoiseq.extractors import MultiSampleVCF
from tqdm import tqdm

In [2]:
import os

try:
    snakemake
except NameError:
    from snakemk_util import load_rule_args
    snakemake = load_rule_args(
        snakefile = os.getcwd() + '/../Snakefile',
        rule_name = 'prioritize_vep_variants',
        root=os.getcwd() + "/..",
        default_wildcards={'chrom': 'chr10'}
    )

In [3]:
df_metadata = pd.read_csv(snakemake.input['released_files'])

In [4]:
df_results = pd.read_csv(snakemake.input['results']).set_index('geneID')
df_genes = pd.read_csv(snakemake.input['genes']).set_index('geneID')
df_results = df_results.join(df_genes)
df_results

,sampleID,pValue,padjust,zScore,l2fc,rawcounts,normcounts,meanCorrected,theta,aberrant,AberrantBySample,AberrantByGene,padj_rank,Chromosome,Start,End,Strand,gene_name,gene_biotype
geneID,,,,,,,,,,,,,,,,,,,
ENSG00000000003,CASE.NEUAA599TMX,7.156284e-18,1.363858e-12,-9.53,-4.51,30,40.08,934.62,19.47,True,5,3,1.0,X,100627108,100639991,-,TSPAN6,protein_coding
ENSG00000000003,CTRL.NEUHE723FGT,6.700160e-09,1.276929e-03,-5.45,-2.59,77,154.18,934.62,19.47,True,5,3,1.0,X,100627108,100639991,-,TSPAN6,protein_coding
ENSG00000000003,CASE.NEUBA169GXD,4.492763e-08,2.140598e-03,-4.95,-2.36,140,181.84,934.62,19.47,True,7,3,4.0,X,100627108,100639991,-,TSPAN6,protein_coding
ENSG00000003056,CTRL.NEUCV136DHM,4.420148e-12,3.240001e-08,6.50,0.46,3296,2594.60,1884.82,498.90,True,103,1,26.0,12,8940362,8949955,-,M6PR,protein_coding
ENSG00000004455,CASE.NEUGW340YEB,4.311222e-12,8.216409e-07,-7.20,-0.75,942,817.30,1374.34,250.69,True,7,1,1.0,1,33007939,33080996,-,AK2,protein_coding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000278637,CASE.NEUJA217MTJ,9.435958e-09,1.123952e-04,-6.70,-2.50,120,64.15,367.32,19.69,True,68,1,15.5,6,26021678,26021990,+,HIST1H4A,protein_coding
ENSG00000278771,CASE.NEUGW340YEB,4.798981e-10,4.572995e-05,5.05,1.96,4043,1815.72,467.70,12.47,True,7,1,2.0,14,49853615,49853914,-,Metazoa_SRP,misc_RNA
ENSG00000279203,CASE.NEUZP278MR4,2.439758e-09,1.937391e-05,4.83,1.13,268,126.04,57.39,60.20,True,33,1,23.5,19,15383201,15383832,-,AC005785.5,TEC


In [5]:
df_results = df_results[df_results['Chromosome'] == snakemake.wildcards['chrom'].replace('chr', '')]

In [6]:
df = pd.read_parquet(snakemake.input['vep'])

In [7]:
df = df[df['AF'].fillna(0) < 0.0001] # subset outliers
print(df.shape)
df = df[df['IMPACT'].isin({'HIGH', 'MODERATE'})] # subset for high impact variants
print(df.shape)
df = df[df['Gene'].isin(set(df_results.index))] # subset outliers
print(df.shape)

(1780776, 89)
(4498, 89)
(151, 89)


In [8]:
vcf = MultiSampleVCF(snakemake.input['vcf'])

In [18]:
df_metadata

,Participant_ID,omic,CGND_ID,iPSC_Line,line,differentiation,data_level,experiment,path,size
0,CASE-NEUAA599TMX,genomics,CGND-HDA-00461,NaN,NaN,NaN,2,WGS,genomics/2_bam/CASE-NEUAA599TMX/CASE-NEUAA599T...,23079388468
1,CASE-NEUAA599TMX,genomics,CGND-HDA-00461,NaN,NaN,NaN,2,WGS,genomics/2_bam/CASE-NEUAA599TMX/CASE-NEUAA599T...,1582288
2,CASE-NEUAB000NKC,genomics,CGND-HDA-00583,NaN,NaN,NaN,2,WGS,genomics/2_bam/CASE-NEUAB000NKC/CASE-NEUAB000N...,22547466160
3,CASE-NEUAB000NKC,genomics,CGND-HDA-00583,NaN,NaN,NaN,2,WGS,genomics/2_bam/CASE-NEUAB000NKC/CASE-NEUAB000N...,1684005
4,CASE-NEUAE993EPR,genomics,CGND-HDA-03249,NaN,NaN,NaN,2,WGS,genomics/2_bam/CASE-NEUAE993EPR/CASE-NEUAE993E...,15054503804
...,...,...,...,...,...,...,...,...,...,...
4777,CTRL-NEURJ861MMD,proteomics,NaN,NaN,NaN,NaN,4,NaN,proteomics/4_matrix/ProteinLevel_v4-1_release_...,5143400
4778,CTRL-NEUUV825HYF,proteomics,NaN,NaN,NaN,NaN,4,NaN,proteomics/4_matrix/ProteinLevel_v4-1_release_...,5143400
4779,CTRL-NEUVZ050YX7,proteomics,NaN,NaN,NaN,NaN,4,NaN,proteomics/4_matrix/ProteinLevel_v4-1_release_...,5143400
4780,CTRL-NEUWT164JRQ,proteomics,NaN,NaN,NaN,NaN,4,NaN,proteomics/4_matrix/ProteinLevel_v4-1_release_...,5143400


In [20]:
sample_mapping = {
    row['CGND_ID']: row['Participant_ID']
    for i, row in df_metadata[df_metadata['omic'] == 'genomics'].iterrows()
}

In [22]:
next(vcf)

Variant(chrom='chr1', pos=16378, ref='T', alt='C', id='None',...)

In [24]:
df['variant'] = snakemake.wildcards['chrom'] + ':' + df['pos'].astype(str) + ':'+ df['ref'] + '>' + df['alt']

In [34]:
df

,pos,ref,alt,Gene,Feature,Feature_type,Consequence,cDNA_position,CDS_position,Protein_position,...,miRNA83,CADD_PHRED,CADD_RAW,condel_prediction,condel_score,sift_prediction,sift_score,polyphen_prediction,polyphen_score,variant
15992,5768222,G,A,ENSG00000057608,ENST00000380181,Transcript,stop_gained,1000/1408,847/1203,283/400,...,None,42.000000,8.330135,None,NaN,None,NaN,None,NaN,chr10:5768222:G>A
15993,5768222,G,A,ENSG00000057608,ENST00000380191,Transcript,stop_gained,1158/2297,982/1338,328/445,...,None,42.000000,8.330135,None,NaN,None,NaN,None,NaN,chr10:5768222:G>A
15994,5768222,G,A,ENSG00000057608,ENST00000447751,Transcript,stop_gained,466/834,466/612,156/203,...,None,42.000000,8.330135,None,NaN,None,NaN,None,NaN,chr10:5768222:G>A
16310,5773889,T,C,ENSG00000057608,ENST00000380181,Transcript,missense_variant,790/1408,637/1203,213/400,...,None,19.770000,2.067882,neutral,0.005,tolerated,0.63,benign,0.031,chr10:5773889:T>C
16311,5773889,T,C,ENSG00000057608,ENST00000380191,Transcript,missense_variant,948/2297,772/1338,258/445,...,None,19.770000,2.067882,neutral,0.006,tolerated,0.63,benign,0.043,chr10:5773889:T>C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3867207,124405471,C,T,ENSG00000065154,ENST00000368845,Transcript,missense_variant,693/2039,613/1320,205/439,...,None,19.059999,1.974205,neutral,0.029,tolerated,0.29,benign,0.003,chr10:124405471:C>T
3867214,124405471,C,T,ENSG00000065154,ENST00000539214,Transcript,missense_variant,518/1864,199/906,67/301,...,None,19.059999,1.974205,neutral,0.027,tolerated,0.31,benign,0.003,chr10:124405471:C>T
3868536,125839128,T,G,ENSG00000089876,ENST00000284690,Transcript,missense_variant,2222/3050,1754/2232,585/743,...,None,21.799999,2.337842,neutral,0.300,tolerated,0.09,benign,0.054,chr10:125839128:T>G
3868537,125839128,T,G,ENSG00000089876,ENST00000368721,Transcript,missense_variant,1561/2389,626/1104,209/367,...,None,21.799999,2.337842,neutral,0.349,deleterious,0.05,benign,0.054,chr10:125839128:T>G


In [39]:
df['samples'] = [
    [
        sample_mapping[k.replace('-b38', '')]
        for k, v in vcf.get_samples(vcf.get_variant(i)).items()
    ]
    for i in df['variant']
]

In [48]:
df = df[df['samples'].map(len) < 7].explode('samples')

In [50]:
df.to_csv(snakemake.output['prioritized'], index=False)